In [13]:
#imports
import os, shutil, fnmatch, time, tkinter
from tkinter.filedialog import askdirectory
from pathlib import Path

In [14]:
#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
path_input = askdirectory(title='Select Folder')  # shows dialog box and return the path
print(path_input)
tk_root.destroy()

E:/Code_processed_eclipse_data/20221208_NHDFneo_P6_IrradECM/Sorted/tissue_ai


In [15]:
stage_pos_chars = []
for file in os.listdir(path_input):
    if file.endswith('.nd'):
        basename = file.split('.',1)[0]
        with open(path_input + '/' +file, 'r') as nd_file:
            for l_no, line in enumerate(nd_file):
                if '"NStagePositions"' in line:
                    stage_pos_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"NTimePoints"' in line:
                    timepoints_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"Stage' in line:
                    spc = line.split(', "')[-1]
                    stage_pos_chars.append(spc[0:len(spc)-2])


In [16]:
for file in os.listdir(path_input):
    if os.path.isdir(path_input + '/' + file):
        #rename file for correct sort syntax
                folder = file.split('s', 1)[-1]

                for num in range(2-len(folder)):
                    folder = '0'+folder
                newfile = 's'+folder
                os.rename(path_input + '/' + file, path_input + '/' + newfile)


In [24]:
import pandas as pd
list_pos= []
tlist = [T*0.5 for T in range(1,timepoints_nd+1)]
wa_df = pd.DataFrame({'Frame': range(1,timepoints_nd+1),
                      'Time':  tlist})
for file in os.listdir(path_input):
    if os.path.isdir(path_input+'/'+file):

        list_pos.append(file)
        wa_df[file] = pd.read_table(path_input+'/'+file+'/segment_ph1/is_closed_vs_frame.txt',header = None)

header1 = wa_df.columns
header2 = ['t','Hours']+ stage_pos_chars
wa_df.columns = [header1, header2]
display(wa_df)

xls_name = path_input.split('Code_processed_eclipse_data/',1)[-1].split('/',1)[0]
#wa_df.to_excel(r'E:/Microscope Images/Code_processed_eclipse_data/Wound_area_code_output.xlsx', sheet_name=xls_name+'_'+ basename)

def append_to_excel(fpath, df, sheet_name):
    with pd.ExcelWriter(fpath, mode="a") as f:
        df.to_excel(f, sheet_name=sheet_name)

append_to_excel('F:/Microscope Images/Code_processed_eclipse_data/Wound_area_code_output.xlsx', wa_df,xls_name+"_cl")


,Frame,Time,s01,s02,s03,s04,s05,s06,s07,s08,...,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54
,t,Hours,A4,A5,A6,A9,A11,b11,B10,B4,...,G1,H2,H4,G4,H5,G6,G7,G8,G9,H12
0,1,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,2.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,3.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,4.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


C:\Users\avasan\Anaconda3\envs\wound-compute-env\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
